In [ ]:
import xarray as xr
import glob

In [ ]:
pth = 'layers/'

In [ ]:
# Load layers, save a copy as tif, convert tif to epsg=4326 with rio commandline

in_paths = pth+f'fmc_nsw_*.nc' # path to tile-combined layers
# in_paths = pth+f'layers/tiles/*/*.nc'
in_paths = glob.glob(in_paths)
for path in in_paths[:1]: # TEMP slice
    print(path)
    # Open dataset
    ds = xr.open_dataset(path)
    # Location for output
    out_pth = path[:-3]+'.tif'
    print(out_pth)
    ds.rio.to_raster(out_pth,compress='ZSTD',predictor=2,zlevel=9)
    
    # Location for output from rio warp in shell
    rio_out = out_pth[:-4]+'_30m_4283.tif'
    print(rio_out)
    # Reproject tif to 30m res, CRS=4283, using rio in shell, use a template tif for formatting using --like
    !rio warp {out_pth} {rio_out} --like data/lf_dem1nNV.tif --resampling nearest --co COMPRESS=ZSTD --co PREDICTOR=2 --co ZLEVEL=9

In [ ]:
# Check outputs

in_paths = pth+f'tiles/*/*.nc'
in_paths = glob.glob(in_paths)
for path in in_paths[:1]:
    print(path)
    out_pth = path[:-3]+'.tif'
    print(out_pth)
    da = xr.open_dataarray(out_pth,chunks='auto').drop('band').squeeze('band')
    da.isel(x=slice(0,500),y=slice(0,500)).plot();plt.show()
    rio_out = out_pth[:-4]+'_30m_4283.tif'
    print(rio_out)
    da = xr.open_dataarray(rio_out,chunks='auto').drop('band').squeeze('band')
    da.isel(x=slice(0,500),y=slice(0,500)).plot();plt.show()